In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv('gs://hkanjih-spark-udemy/material/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv', header=True, inferSchema=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [9]:
training, test = data.randomSplit([0.8,0.2])

In [10]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId',itemCol='movieId',ratingCol='rating')

In [12]:
model = als.fit(training)

In [14]:
predictions = model.transform(test)

In [15]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|     5|  0.6401748|
|     85|   3.0|     1|0.016855802|
|     85|   1.0|    13|  1.5966246|
|     85|   1.0|     4|   3.286133|
|     85|   1.0|    23| -2.8038297|
|     65|   2.0|     5|  1.5010636|
|     65|   1.0|    19|  1.6277544|
|     65|   2.0|    15| 0.29030854|
|     53|   3.0|    20| -1.9571674|
|     53|   1.0|     9|  2.0926242|
|     78|   1.0|     2| 0.27977726|
|     34|   1.0|    17|  0.4323245|
|     34|   3.0|    25| 0.33500594|
|     81|   1.0|     1| 0.17346579|
|     81|   2.0|    29|  1.1631229|
|     81|   3.0|    18| -0.6756648|
|     28|   1.0|    27|  2.5599906|
|     76|   3.0|     3| -2.0374718|
|     76|   1.0|    20| -0.9776079|
|     26|   1.0|    22| 0.42672035|
+-------+------+------+-----------+
only showing top 20 rows



In [16]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [18]:
rmse = evaluator.evaluate(predictions)

In [19]:
print('RMSE')
print(rmse)

RMSE
1.80451950369


In [20]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [21]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     20|    11|
|     21|    11|
|     25|    11|
|     32|    11|
|     39|    11|
|     41|    11|
|     59|    11|
|     61|    11|
|     64|    11|
|     71|    11|
+-------+------+



In [22]:
recommendation = model.transform(single_user)

In [23]:
recommendation.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     59|    11|  3.9651792|
|     39|    11|  3.9347625|
|     32|    11|   3.405784|
|     71|    11|  1.9496776|
|     20|    11|  1.8792393|
|     21|    11|  1.2765756|
|     64|    11|  1.0874891|
|     41|    11|  0.8195978|
|     61|    11|-0.28448942|
|     25|    11| -4.1741877|
+-------+------+-----------+

